In [1]:
setwd("~/Projects/livemanager")
load("plm.RData")

In [2]:
odds <- read.csv("data/match_odds.csv")
next_odds <- read.csv("data/next_match_odds.csv")
str(odds)

'data.frame':	234 obs. of  18 variables:
 $ LEAGUE   : Factor w/ 1 level "Bundesliga": 1 1 1 1 1 1 1 1 1 1 ...
 $ FTR      : Factor w/ 3 levels "A","D","H": 3 3 3 3 2 1 3 2 3 3 ...
 $ FTSC     : Factor w/ 28 levels "0:0","0:1","0:2",..: 12 6 25 17 14 3 6 20 12 12 ...
 $ MATCH_URL: Factor w/ 234 levels "http://www.oddsportal.com/soccer/germany/bundesliga/augsburg-b-monchengladbach-GncNw7cb/",..: 77 33 52 18 54 114 144 155 128 100 ...
 $ SEASON   : int  2015 2015 2015 2015 2015 2015 2015 2015 2015 2015 ...
 $ HOMETEAM : Factor w/ 18 levels "1. FC Koln","Augsburg",..: 7 4 5 3 6 10 12 13 11 9 ...
 $ TIME     : Factor w/ 10 levels "13:30","14:30",..: 5 2 6 2 2 2 2 2 10 5 ...
 $ FTHG     : int  2 1 5 3 2 0 1 3 2 2 ...
 $ AVOD     : num  5.14 3.97 11 4.55 3.18 3.31 3.7 3.31 3.24 3.15 ...
 $ AWAYTEAM : Factor w/ 18 levels "1. FC Koln","Augsburg",..: 14 9 17 8 2 1 18 16 15 11 ...
 $ AVOA     : num  8.85 5.44 24.43 6.21 2.75 ...
 $ COUNTRY  : Factor w/ 1 level "Germany": 1 1 1 1 1 1 1 1 1 1 ...


In [3]:
## Express Odds as Chances
next_odds$Hprob <- (1 / next_odds$AVOH) / ((1/next_odds$AVOH) + (1/next_odds$AVOD) + (1/next_odds$AVOA))
next_odds$Aprob <- (1 / next_odds$AVOA) / ((1/next_odds$AVOH) + (1/next_odds$AVOD) + (1/next_odds$AVOA))
odds$Hprob <- (1 / odds$AVOH) / ((1/odds$AVOH) + (1/odds$AVOD) + (1/odds$AVOA))
odds$Aprob <- (1 / odds$AVOA) / ((1/odds$AVOH) + (1/odds$AVOD) + (1/odds$AVOA))


In [4]:
#next_odds
#head(odds)
levels(odds$HOMETEAM)

[1] "1. FC Koln"          "Augsburg"            "B. Monchengladbach" 
 [4] "Bayer Leverkusen"    "Bayern Munich"       "Darmstadt"          
 [7] "Dortmund"            "Eintracht Frankfurt" "Hamburger SV"       
[10] "Hannover"            "Hertha Berlin"       "Hoffenheim"         
[13] "Ingolstadt"          "Mainz"               "Schalke"            
[16] "VfB Stuttgart"       "Werder Bremen"       "Wolfsburg"

In [5]:
## Align Teamnames with PLM format
levels(next_odds$HOMETEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")
levels(next_odds$AWAYTEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")
levels(odds$HOMETEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")
levels(odds$AWAYTEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")

## Add matchday column
matchdays <- nrow(odds) / 9
odds$matchday <- rep(matchdays:1, each = 9)
furthest_matchday <- (nrow(next_odds) / 9) + matchdays
next_odds$matchday <- rep((matchdays+1):furthest_matchday, each = 9)

In [6]:
sqldf("SELECT timestamp, DATE, COUNT(1) FROM odds GROUP BY timestamp, DATE ORDER BY timestamp")
#odds[80:120,]
#rep(matchdays:1, each = 9)
#odds$matchday

ERROR: Error in eval(expr, envir, enclos): could not find function "sqldf"


In [7]:
levels(plm$home_shortname) <- factor(plm$Club)
levels(plm$away_shortname) <- factor(plm$Club)

In [8]:
## Get average earnings per team / match
library("dplyr")
plm.clubmatches <- plm %>%
    filter(!is.na(Club) & time_on_pitch > 0) %>%
    group_by(matchday, Club) %>%
    summarise(home = ceiling(mean(Club == home_shortname)),
              earnings = mean(total_earnings)) %>%
    ungroup() %>%
    arrange(matchday, Club)

head(plm.clubmatches, 20)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



,matchday,Club,home,earnings
1,1,B04,0,129208.3
2,1,BMG,0,-692.3077
3,1,BRE,0,9321.429
4,1,BSC,NA,109846.2
5,1,BVB,0,186961.5
6,1,D98,0,62642.86
7,1,FCA,NA,19607.14
8,1,FCB,NA,203142.9
9,1,FCI,NA,109285.7
10,1,H96,0,66928.57


In [10]:
#odds['matchday']

probs.home <- odds[c('matchday', 'HOMETEAM', 'Hprob')]
probs.away <- odds[c('matchday', 'AWAYTEAM', 'Aprob')]
names <- c("matchday", "Club", "Prob")
colnames(probs.home) <- names
colnames(probs.away) <- names
probs <- rbind(probs.home, probs.away)
subset(probs, probs$matchday == 9)

,matchday,Club,Prob
154,9,VFB,0.5177454
155,9,KOE,0.5230991
156,9,SGE,0.3701812
157,9,FCA,0.5125797
158,9,HSV,0.2209256
159,9,S04,0.4899106
160,9,BRE,0.06434458
161,9,WOB,0.5974875
162,9,M05,0.1524443
388,9,FCI,0.2275128


In [12]:
library("sqldf")

Loading required package: gsubfn
Loading required package: proto
Warning message:
In doTryCatch(return(expr), name, parentenv, handler): unable to load shared object '/home/pi/anaconda2/lib64/R/modules//R_X11.so':
  libSM.so.6: cannot open shared object file: No such file or directoryCould not load tcltk.  Will use slower R code instead.
Loading required package: RSQLite
Loading required package: DBI


In [25]:
clubmatches <- plm.clubmatches

earnings_probs <- sqldf("SELECT t1.matchday AS matchday, t1.Club AS Club, home, earnings AS team_earnings, Prob
                         FROM clubmatches t1 JOIN probs t2
                         ON t1.matchday = t2.matchday AND t1.Club = t2.Club")
#earnings_probs <- merge(plm.clubmatches, probs, by = c("matchday", "Club"))
subset(earnings_probs, earnings_probs$matchday == 26)

,matchday,Club,home,team_earnings,Prob
451,26,B04,0,100458.3,0.583237
452,26,BMG,0,148678.6,0.6349108
453,26,BRE,NA,-22100,0.03932363
454,26,BSC,0,140458.3,0.375894
455,26,BVB,0,155041.7,0.7050875
456,26,D98,0,77772.73,0.349067
457,26,FCA,0,72625,0.349067
458,26,FCB,NA,202423.1,0.8733422
459,26,FCI,NA,78590.91,0.3149976
460,26,H96,NA,29055.56,0.2815953


In [14]:
library("ggplot2")
ggplot(data = earnings_probs, aes(x = Prob, y = earnings)) +
geom_point() +
geom_smooth()

ERROR: Error in file(con, "rb"): cannot open the connection


ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

In [40]:
## Add prob at individual player level
plm.merge <- merge(plm, earnings_probs, by = c("matchday", "Club"), all.x = TRUE)

In [28]:
subset(plm.merge, plm.merge$matchday == 26)

,matchday,Club,id,assist,clean_sheet,goal,game_play_duration,total_earnings,away_shortname,away_squad,away_score,time_on_pitch,status,match_id,period,finished,home_shortname,home_squad,home_score,in_for_player,sub_in_timestamp,sub_in_minutes,sub_out_minutes,sub_out_timestamp,shot_on_target,attempt_saved,successful_pass,Name,init_Value,Pos,home,team_earnings,Prob
14951,26,B04,64892,0,NA,0,0,0,NA,1572,0,0,bench,8685,FullTime,True,KOE,1564,1,NA,NA,NA,NA,NA,0,NA,NA,A. Mehmedi,6.5,ATT,0,100458.3,0.583237
14952,26,B04,64511,0,1,0,90,55500,NA,1572,0,90,red_card,8685,FullTime,True,KOE,1564,1,NA,1457879400,0,96,1457886126,NA,NA,NA,T. Jedvaj,8,DEF,0,100458.3,0.583237
14953,26,B04,64998,0,0,0,0,0,NA,1572,0,0,bench,8685,FullTime,True,KOE,1564,1,NA,NA,NA,NA,NA,NA,NA,NA,B. Henrichs,5,MID,0,100458.3,0.583237
14954,26,B04,64889,0,0,0,13,35000,NA,1572,0,13,sub_in,8685,FullTime,True,KOE,1564,1,64997,1457885031,77,NA,NA,NA,NA,NA,A. Ramalho,7.5,DEF,0,100458.3,0.583237
14955,26,B04,64510,0,1,0,90,185500,NA,1572,0,90,starter,8685,FullTime,True,KOE,1564,1,NA,1457879400,0,NA,NA,NA,NA,NA,K. Papadopoulos,8.5,DEF,0,100458.3,0.583237
14956,26,B04,64509,0,0,0,0,0,NA,1572,0,0,not_present,8685,FullTime,True,KOE,1564,1,NA,NA,NA,NA,NA,NA,NA,NA,R. Hilbert,10,DEF,0,100458.3,0.583237
14957,26,B04,64519,0,1,0,90,106000,NA,1572,0,90,starter,8685,FullTime,True,KOE,1564,1,NA,1457879400,0,NA,NA,NA,NA,NA,J. Brandt,7,MID,0,100458.3,0.583237
14958,26,B04,64545,0,0,0,0,0,NA,1572,0,0,not_present,8685,FullTime,True,KOE,1564,1,NA,NA,NA,NA,NA,NA,NA,NA,K. Kampl,6.5,MID,0,100458.3,0.583237
14959,26,B04,64515,0,0,0,0,0,NA,1563,0,0,not_present,8686,FullTime,True,KOE,1565,2,NA,NA,NA,NA,NA,NA,NA,NA,G. Donati,6,DEF,0,100458.3,0.583237
14960,26,B04,64518,0,1,0,90,110500,NA,1572,0,90,starter,8685,FullTime,True,KOE,1564,1,NA,1457879400,0,NA,NA,NA,NA,NA,H. Calhanoglu,11,MID,0,100458.3,0.583237


In [16]:
#options(jupyter.plot_mimetypes = 'image/png')

ggplot(data = subset(plm.merge, plm.merge$time_on_pitch > 0), aes(x=Prob, y = total_earnings)) +
geom_point() +
geom_smooth()

Warning message:
: Removed 143 rows containing non-finite values (stat_smooth).Warning message:
: Removed 143 rows containing missing values (geom_point).

ERROR: Error in file(con, "rb"): cannot open the connection


ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

## Regression model to predict earnings

In [41]:
load("starter_probs.RData")
plm.merge <- merge(plm.merge, starter_probs, by = c("id", "matchday") )

In [42]:
colnames(plm.merge)[ncol(plm.merge)] <- c("probStarter")
subset(plm.merge, plm.merge$id == 64467)

,id,matchday,Club,assist,clean_sheet,goal,game_play_duration,total_earnings,away_shortname,away_squad,away_score,time_on_pitch,status,match_id,period,finished,home_shortname,home_squad,home_score,in_for_player,sub_in_timestamp,sub_in_minutes,sub_out_minutes,sub_out_timestamp,shot_on_target,attempt_saved,successful_pass,Name,init_Value,Pos,home,team_earnings,Prob,0,probStarter
1,64467,10,KOE,NA,0,NA,90,35000,NA,1562,0,90,starter,8851,FullTime,True,NA,1569,4,NA,1445693400,0,NA,NA,NA,9,25,T. Horn,8.5,GOA,NA,-12428.57,0.04887098,0.001998002,0.998002
2,64467,11,KOE,NA,1,NA,90,128500,NA,1577,0,90,starter,8838,FullTime,True,NA,1562,0,NA,1446301800,0,NA,NA,NA,4,35,T. Horn,8.5,GOA,NA,86923.08,0.4388674,0.000999001,0.999001
3,64467,12,KOE,NA,0,NA,90,82500,NA,1562,2,90,starter,8866,FullTime,True,KOE,1564,1,NA,1446906600,0,NA,NA,NA,3,23,T. Horn,8.5,GOA,1,94153.85,0.1563186,0,1
4,64467,13,KOE,NA,1,NA,90,116500,NA,1563,0,90,starter,8856,FullTime,True,NA,1562,0,NA,1448116200,0,NA,NA,NA,1,26,T. Horn,8.5,GOA,NA,80500,0.445153,0,1
5,64467,14,KOE,NA,1,NA,90,126000,NA,1562,0,90,starter,8889,FullTime,True,KOE,1575,0,NA,1448652600,0,NA,NA,NA,1,23,T. Horn,8.5,GOA,1,78142.86,0.3889625,0,1
6,64467,15,KOE,NA,0,NA,90,57000,NA,1568,1,90,starter,8874,FullTime,True,NA,1562,0,NA,1449325800,0,NA,NA,NA,7,31,T. Horn,8.5,GOA,NA,23964.29,0.4198473,0,1
7,64467,16,KOE,NA,0,NA,90,131000,NA,1562,1,90,starter,8907,FullTime,True,KOE,1576,1,NA,1449930600,0,NA,NA,NA,5,23,T. Horn,8.5,GOA,1,67076.92,0.3525251,0,1
8,64467,17,KOE,NA,0,NA,90,66000,KOE,1565,1,90,starter,8892,FullTime,True,NA,1562,2,NA,1450535400,0,NA,NA,NA,2,15,T. Horn,8.5,GOA,NA,87115.38,0.1591019,0,1
9,64467,18,KOE,NA,0,NA,90,28500,NA,1578,3,90,starter,8919,FullTime,True,NA,1562,1,NA,1453559400,0,NA,NA,NA,5,29,T. Horn,8.5,GOA,NA,32769.23,0.4584521,0,1
10,64467,19,KOE,NA,0,NA,90,49000,NA,1562,1,90,starter,8918,FullTime,True,NA,1579,1,NA,1454250600,0,NA,NA,NA,4,16,T. Horn,8.5,GOA,NA,61500,0.1725836,0,1


In [43]:
fit <- lm(total_earnings ~ Prob + init_Value + Pos + probStarter, data = plm.merge)
summary(fit)

## Double-check merges are correct
## R-Squared of 0.29; add running average of earnings (needs to be previous weeks' to serve as predictor).


Call:
lm(formula = total_earnings ~ Prob + init_Value + Pos + probStarter, 
    data = plm.merge)

Residuals:
    Min      1Q  Median      3Q     Max 
-191145  -31899  -10804   18194  532205 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -21277       2821  -7.544 5.07e-14 ***
Prob           82913       3787  21.892  < 2e-16 ***
init_Value      1928        369   5.225 1.79e-07 ***
PosDEF        -21392       1864 -11.474  < 2e-16 ***
PosGOA        -25854       2939  -8.798  < 2e-16 ***
PosMID         -5530       1774  -3.118  0.00183 ** 
probStarter    86187       1839  46.875  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 56500 on 8045 degrees of freedom
  (5103 observations deleted due to missingness)
Multiple R-squared:  0.2896,	Adjusted R-squared:  0.2891 
F-statistic: 546.6 on 6 and 8045 DF,  p-value: < 2.2e-16
